# Recursive Ekphrasis Gym

## Processing `(adj,noun)` pairs

In [ ]:
! wget https://github.com/kbooten/ekphrasisgym/blob/main/adj2nouns_tuples_with_count.json?raw=true

--2022-08-12 10:47:50--  https://github.com/kbooten/ekphrasisgym/blob/main/adj2nouns_tuples_with_count.json?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/kbooten/ekphrasisgym/raw/main/adj2nouns_tuples_with_count.json [following]
--2022-08-12 10:47:50--  https://github.com/kbooten/ekphrasisgym/raw/main/adj2nouns_tuples_with_count.json
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/kbooten/ekphrasisgym/main/adj2nouns_tuples_with_count.json [following]
--2022-08-12 10:47:50--  https://media.githubusercontent.com/media/kbooten/ekphrasisgym/main/adj2nouns_tuples_with_count.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to media.githubuser

In [ ]:
import json

with open('adj2nouns_tuples_with_count.json?raw=true','r') as f:
  adj2noun_tuples_with_count = json.load(f)

In [ ]:
adj2noun_tuples_with_count[:4]

[[['holy', 'writ', 1], 8],
 [['vulgar', 'thief', 1], 6],
 [['too', 'base', 1], 5],
 [['cold', 'fountain', 1], 10]]

In [ ]:
[n for n,c in adj2noun_tuples_with_count if n[1]=="sherbet"]#.count(1)

[['little', 'sherbet', 1],
 ['rich', 'sherbet', 1],
 ['excellent', 'sherbet', 1],
 ['exquisite', 'sherbet', 1],
 ['turkish', 'sherbet', 1],
 ['iced', 'sherbet', 1],
 ['nice', 'sherbet', 1],
 ['pink', 'sherbet', 1],
 ['cool', 'sherbet', 1],
 ['drunk', 'sherbet', 1]]

Using WordNet to make sure a token is a word.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
wn.synsets("sherbet",pos=wn.NOUN)

[Synset('sherbert.n.01')]

In [ ]:
from collections import defaultdict

noun2adjs = defaultdict(list)

In [ ]:
min_count = 3

for tup,count in adj2noun_tuples_with_count:
  adj,noun,num = tup
  if count>=min_count: ### don't add very rare words
    if (adj.isalpha() and noun.isalpha() and adj.islower() and noun.islower()):
      if wn.synsets(noun,pos=wn.NOUN)!=[]:
        noun_plus_num = noun+"~"+str(num)
        noun2adjs[noun_plus_num]+=[adj]*count

Get rid of values that are the same as the key.

In [ ]:
for key,values in noun2adjs.items():
  noun2adjs[key] = [v for v in values if v.split("~")[0]!=key]

In [ ]:
noun2adjs["pig~1"][:3]

['hungry', 'hungry', 'hungry']

Sometimes a value is just a letter.  Filter out really short words.

In [ ]:
for key,values in noun2adjs.items():
  noun2adjs[key] = [v for v in values if len(v)>2]

### Rank by TF-IDF

In [ ]:
sets_of_words = list(noun2adjs.values())

In [ ]:
total_number_of_sets = len(sets_of_words)
total_number_of_sets

26487

In [ ]:
from collections import defaultdict

word2doc_count = defaultdict(int)

In [ ]:
for s in sets_of_words:
  s = list(set(s))
  for t in s:
    word2doc_count[t]+=1

In [ ]:
word2doc_count['eat']

0

In [ ]:
noun2adjs_and_weights = {}

In [ ]:
for key,words in noun2adjs.items():
  if len(words)!=0: ## no empty sets
    unique_words = list(set(words))
    adjs_and_weights = []
    for w in unique_words:
      tf = words.count(w)/len(words)
      idf = total_number_of_sets/word2doc_count[w]
      tfidf = tf * idf
      adjs_and_weights.append((w,tfidf))
    adjs_and_weights.sort(key=lambda x:x[1], reverse=True)
    noun2adjs_and_weights[key]=adjs_and_weights[:20]

Make dictionary more concise.

```
dict[key] = {
  "num":1, ## or 2
  "adjs":[("fulsome",11.45),...]
}
```

First, only keep singular *or* plural of noun, whichever has more results (more likely to be correct).

In [ ]:
all_keys_without_number = list(set([n.split("~")[0] for n in noun2adjs_and_weights]))

In [ ]:
for k in all_keys_without_number:
  if ((k+"~1" in noun2adjs_and_weights) and (k+"~2" in noun2adjs_and_weights)):
    if len(noun2adjs_and_weights[k+"~1"])>len(noun2adjs_and_weights[k+"~2"]):
      del noun2adjs_and_weights[k+"~2"]
    else:
      del noun2adjs_and_weights[k+"~1"]

In [ ]:
noun2adjs_and_weights_reformatted = {}

for key,values in noun2adjs_and_weights.items():
  # print(key)
  # print(key.split("~"))
  key,num = key.split("~")
  noun2adjs_and_weights_reformatted[key]={}
  noun2adjs_and_weights_reformatted[key]["num"]=num
  noun2adjs_and_weights_reformatted[key]["verbs"]=values

In [ ]:
noun2adjs_and_weights_reformatted["laugh"]

{'num': '1',
 'verbs': [('mirthless', 16.38184961527223),
  ('screechy', 5.15963767410149),
  ('hearty', 3.454865496644985),
  ('hysteric', 3.095782604460894),
  ('forced', 2.8729800685337845),
  ('sneering', 2.8294787245072683),
  ('ringing', 2.735497559976221),
  ('unmirthful', 2.579818837050745),
  ('derisive', 2.013040456183536),
  ('scornful', 1.9997412287198082),
  ('vexed', 1.8058731859355213),
  ('tinkling', 1.7198792247004968),
  ('grating', 1.6868046242254873),
  ('embarrassed', 1.6256392671826614),
  ('rippling', 1.6123867731567156),
  ('maniacal', 1.5765559559754554),
  ('loud', 1.5674406736614133),
  ('infectious', 1.339521319237887),
  ('merry', 1.3359118942217125),
  ('sardonic', 1.329598939095384)]}

In [ ]:
with open('noun2adjs_and_weights.json','w') as f:
  json.dump(noun2adjs_and_weights_reformatted,f)

In [ ]:
from google.colab import files
files.download('/content/noun2adjs_and_weights.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***